## Snowflake Connection

In [1]:
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import Variant
from snowflake.snowpark.version import VERSION

connection_parameters = {
    "account": "BPSYWTN-HEXAWARE",
    "user": "MOHAMMADA1",
    "password": "Paymatix@123!",
    "role": "SYSADMIN",
    "database": "ML_MODELS",
    "schema": "USECASES",
    "warehouse": "PAYMATIX_ML_WH",
}
def snowflake_connector():
    try:
        session = Session.builder.configs(connection_parameters).create()
        print("Connection Successfull!")
    except:
        raise ValueError("error while connecting with db")
    return session
session = snowflake_connector()
session.telemetry_enabled = False

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION
# Current Environment Details
print('Connection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

Connection Successfull!
Connection Established with the following parameters:
User                        : CARDS_DWH_USER
Role                        : "SYSADMIN"
Database                    : "ML_MODELS"
Schema                      : "USECASES"
Warehouse                   : "SNOWPARK_WH"
Snowflake version           : 7.37.2
Snowpark for Python version : 1.8.0


## Loading Raw Data From Local Storage to Snowflake

In [2]:
buf_data = pd.read_csv("BUF_CustomerTable02.csv")
buf_data.columns = buf_data.columns.str.upper()
buf_data

CUSTOMERID      CUSTOMER_NAME  AVERAGE_DAILY_BALANCE  \
0        9996274  Turpis Consulting               10771.09   
1        9996274  Turpis Consulting               10771.09   
2        9996274  Turpis Consulting               10771.09   
3        9996274  Turpis Consulting               10771.09   
4        9996274  Turpis Consulting               10771.09   
...          ...                ...                    ...   
3343     9996324     Vel Mauris LLP                1970.62   
3344     9996324     Vel Mauris LLP                1970.62   
3345     9996324     Vel Mauris LLP                1970.62   
3346     9996324     Vel Mauris LLP                1970.62   
3347     9996324     Vel Mauris LLP                1970.62   

      CURRENT_BALANCE_OF_CUSTOMER DUE_DATE_PAYMENT  TOTAL_PURCHASE_AMOUNT  \
0                        10703.46       15-07-1997                  17.76   
1                        10703.46       15-07-1997                  17.76   
2                        10703.46       15-07-1997                  17.76   
3                        10703.46       15-07-1997                  17.76   
4                        10703.46       15-07-1997                  17.76   
...                           ...              ...                    ...   
3343                      1899.65       15-07-1997                  17.76   
3344                      1899.65       15-07-1997                  17.76   
3345                      1899.65       15-07-1997                  17.76   
3346                      1899.65       15-07-1997                  17.76   
3347                      1899.65       15-07-1997                  17.76   

      CREDIT_LIMIT CREDIT_LIMIT_DATE  INCOME_OF_THE_CUSTOMER  \
0             21.2        19-01-2022                   10107   
1            938.1        30-07-2021                   10107   
2            365.7        15-11-2021                   10107   
3            121.9        31-12-2021                   10107   
4             26.5        18-01-2022                   10107   
...            ...               ...                     ...   
3343         869.2        12-08-2021                   10059   
3344         879.8        10-08-2021                   10059   
3345         885.1        09-08-2021                   10059   
3346         916.9        03-08-2021                   10059   
3347         927.5        01-08-2021                   10059   

      CURRENTBANKBRANCHID  ...       PHONE MONTHOFBIRTH_OF_CUSTOMER  \
0                     139  ...  9999883143                  October   
1                     139  ...  9999883143                  October   
2                     139  ...  9999883143                  October   
3                     139  ...  9999883143                  October   
4                     139  ...  9999883143                  October   
...                   ...  ...         ...                      ...   
3343                  129  ...  9999883093                  October   
3344                  129  ...  9999883093                  October   
3345                  129  ...  9999883093                  October   
3346                  129  ...  9999883093                  October   
3347                  129  ...  9999883093                  October   

      YEAROFBIRTH_OF_CUSTOMER BILLING_ADDRESS_LINE_1  BILLING_CITY  \
0                        1994      150_Gold_Star_Hwy        Groton   
1                        1994      150_Gold_Star_Hwy        Groton   
2                        1994      150_Gold_Star_Hwy        Groton   
3                        1994      150_Gold_Star_Hwy        Groton   
4                        1994      150_Gold_Star_Hwy        Groton   
...                       ...                    ...           ...   
3343                     1948           27520_Hwy_98        Daphne   
3344                     1948           27520_Hwy_98        Daphne   
3345                     1948           27520_Hwy_98        Daphne   
3346          

In [3]:
# creating snowflake dataframe
buf_snow = session.createDataFrame(buf_data) #data=buf_data, schema=schema_buf)
buf_snow.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMERID"  |"CUSTOMER_NAME"    |"AVERAGE_DAILY_BALANCE"  |"CURRENT_BALANCE_OF_CUSTOMER"  |"DUE_DATE_PAYMENT"  |"TOTAL_PURCHASE_AMOUNT"  |"CREDIT_LIMIT"  |"CREDIT_LIMIT_DATE"  |"INCOME_OF_THE_CUSTOMER"  |"CURRENTBANKBRANCHID"  |"CUSTOMERMONTHSONBOOKS"  |"STARTING_DATE_OF_RELATIONSHIP"  |"AGE"  |"EMAIL_ADDRESS"       |"PHONE"     |"MONTHOFBIRTH_OF_CUSTOMER"  |"YEAROFBIRTH_OF_CUSTOMER"  |"BILLING_ADDRESS_LINE_1"  |"BILLING_CITY"  |"BILLING_STATE"  |"BILLING_COUNTR

In [4]:
buf_snow.write.mode("overwrite").save_as_table("BUF_CUSTOMER_TABLE") # writing snowflake dataframe to snowflake table
# buf = session.sql("select * from BUF_CUSTOMER_TABLE")  ## Snowlake DataFrame
buf = session.table("BUF_CUSTOMER_TABLE").to_pandas()  ## Converts Snowflake.Table to Pandas DataFrame
buf.head(3)

CUSTOMERID      CUSTOMER_NAME  AVERAGE_DAILY_BALANCE  \
0     9996274  Turpis Consulting               10771.09   
1     9996274  Turpis Consulting               10771.09   
2     9996274  Turpis Consulting               10771.09   

   CURRENT_BALANCE_OF_CUSTOMER DUE_DATE_PAYMENT  TOTAL_PURCHASE_AMOUNT  \
0                     10703.46       15-07-1997                  17.76   
1                     10703.46       15-07-1997                  17.76   
2                     10703.46       15-07-1997                  17.76   

   CREDIT_LIMIT CREDIT_LIMIT_DATE  INCOME_OF_THE_CUSTOMER  \
0          21.2        19-01-2022                   10107   
1         938.1        30-07-2021                   10107   
2         365.7        15-11-2021                   10107   

   CURRENTBANKBRANCHID  ...       PHONE MONTHOFBIRTH_OF_CUSTOMER  \
0                  139  ...  9999883143                  October   
1                  139  ...  9999883143                  October   
2                  139  ...  9999883143                  October   

   YEAROFBIRTH_OF_CUSTOMER BILLING_ADDRESS_LINE_1  BILLING_CITY BILLING_STATE  \
0                     1994      150_Gold_Star_Hwy        Groton   Connecticut   
1                     1994      150_Gold_Star_Hwy        Groton   Connecticut   
2                     1994      150_Gold_Star_Hwy        Groton   Connecticut   

   BILLING_COUNTRY BILLING_ZIP CUSTOMERSCORE CUSTOMERSCOREFLAG  
0               US        6340            37         Good Mark  
1               US        6340            37         Good Mark  
2               US        6340            37         Good Mark  

[3 rows x 24 columns]

# Create Stored Procedure to deploy training code in Snowflake
## Execute SP to train model and deploy on Snowflake
Create a Python function that will be executed as a Stored Procedure in Snowflake
<br>Python function to train a model using scikit-learn

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler, FunctionTransformer

from sklearn.ensemble import RandomForestRegressor


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

import dill

import os
import tempfile
import joblib
from snowflake.snowpark.functions import sproc

session.clear_imports()
session.clear_packages()
session.use_schema("USECASES")
session.add_packages("snowflake-snowpark-python", "pandas", "scikit-learn", "dill" ,"joblib")

def data_preprocessing(df: pd.DataFrame, func_type: str, columns: list):
    if func_type == 'date':
        df_date = df[columns].copy()
        df_date[columns] = df_date[columns].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y'))
        for col in columns:
            df_date[col + '_YEAR'] = df_date[col].dt.year
            df_date[col + '_MONTH'] = df_date[col].dt.month
            df_date[col + '_DAY'] = df_date[col].dt.day
        df_date.drop(columns, axis=1, inplace=True)
        return df_date
    elif func_type == 'month':
        month_to_number = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
        df[columns] = df[columns].map(month_to_number)
        return df
    elif func_type == 'category':
        label_encoders = {}
        df_category = df[columns].copy()
        df_category['EMAIL_DOMAIN'] = df_category['EMAIL_ADDRESS'].str.split('@').str[-1]
        df_category['PHONE'] = df_category['PHONE'].apply(lambda x: '91'+str(x))
        df_category['PHONE'] = df_category['PHONE'].str.replace('[^0-9]', '', regex=True)
        df_category['PHONE'] = df_category['PHONE'].str[:2].astype(int)
        df_category.drop(['EMAIL_ADDRESS', 'BILLING_ADDRESS_LINE_1'], axis=1, inplace=True, errors='ignore')
        for column in df_category.columns:
            label_encoders[column] = LabelEncoder()
            df_category[column] = label_encoders[column].fit_transform(df_category[column])
        return df_category
    else:
        print('Invalid func_type provided. Please choose from "date", "month", or "category".')
        return None

@sproc(name='TRAIN_BUF_MODEL', session=session, replace=True, is_permanent=True, stage_location='@ML_MODELS.USECASES.BUF_STAGE') 
def dev_train_buf_model(session: Session, train_table: str, train_accuracy_threshold: float = 20, 
        test_accuracy_threshold: float = 20, save_model: bool = True) -> Variant:
    
    buf = session.table(train_table).to_pandas()
    buf.columns = buf.columns.str.upper()
    buf.drop(columns = ["BILLING_ADDRESS_LINE_1", "CUSTOMERID", "CUSTOMER_NAME", "CUSTOMERSCOREFLAG"], inplace=True, errors='ignore')
    buf.dropna(inplace=True)

    date_columns = ["DUE_DATE_PAYMENT", "CREDIT_LIMIT_DATE", "STARTING_DATE_OF_RELATIONSHIP"]
    category_columns = ["BILLING_CITY", "BILLING_STATE", "BILLING_COUNTRY", "BILLING_ZIP", "EMAIL_ADDRESS", "PHONE"]
    numeric_columns = ["AVERAGE_DAILY_BALANCE", "CURRENT_BALANCE_OF_CUSTOMER", "TOTAL_PURCHASE_AMOUNT", "CREDIT_LIMIT", "INCOME_OF_THE_CUSTOMER", "CURRENTBANKBRANCHID", "CUSTOMERMONTHSONBOOKS", "AGE"]
    month_column = "MONTHOFBIRTH_OF_CUSTOMER"
    year_column = "YEAROFBIRTH_OF_CUSTOMER"

    target = "CUSTOMERSCORE"
    
    date_pipeline = Pipeline([
        ('date_preprocessing', FunctionTransformer(data_preprocessing, kw_args={'func_type': 'date' , 'columns': date_columns})),
    ])
    month_pipeline = Pipeline([
        ('map_month_to_number', FunctionTransformer(data_preprocessing, kw_args={'func_type': 'month' , 'columns': month_column})),
    ])
    category_pipeline = Pipeline([
        ('category_preprocessing', FunctionTransformer(data_preprocessing, kw_args={'func_type': 'category' , 'columns': category_columns})),
    ])
    preprocessor = ColumnTransformer(transformers=[
        ('date_pipeline', date_pipeline, date_columns),
        ('month_pipeline', month_pipeline, [month_column]),
        ('category_pipeline', category_pipeline, category_columns)
        ],
        remainder='passthrough',
        n_jobs=-1
    )    
        
    model = Pipeline(steps=[
        ('data-preprocessor', preprocessor),
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor(n_estimators=216, max_depth=10, max_leaf_nodes=100 ,random_state=42))
    ], verbose=True)

    ######################### Dataset Splitting for Training and Testing #########################
    X = buf[date_columns+category_columns+numeric_columns+[month_column, year_column]]
    y = buf[target]
    # print('Input Feartures Required: ', list(X.columns), '\n', f'Total Features: {len(X.columns)}')
    X_train, X_test, y_train, y_test =  train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    ######################### Model Training #########################
    model.fit(X_train, y_train)
    # print(model['regressor'].feature_importances_)
    
    # Make predictions using the trained model
    train_predictions = model.predict(X_train)
    train_r2_score = r2_score(y_train, train_predictions)
    test_predictions = model.predict(X_test)
    test_r2_score = r2_score(y_test, test_predictions)

    model_saved = False
    session_put = None
    if save_model:
        if train_r2_score >= train_accuracy_threshold and test_r2_score>= test_accuracy_threshold:
            # Upload the model to a stage
                try:
                    # with tempfile.TemporaryDirectory() as model_output_dir:
                    model_output_dir = 'tmp'
                    os.makedirs(model_output_dir, exist_ok=True)
                    print(model_output_dir)
                    model_name = os.path.join(model_output_dir, "buf_model.joblib.gz")
                    # joblib.dump(model, model_name)
                    with open(model_name, 'wb') as model_file:
                        dill.dump(model, model_file)
                    session_put = session.file.put(model_name, '@ML_MODELS.USECASES.BUF_STAGE', overwrite=True)
                    model_saved = True
                except Exception as e:
                    print(e)
                    raise e
    return {
        "r2 score on train": train_r2_score,
        "r2 threshold on train": train_accuracy_threshold,
        "r2 score on teet": test_r2_score,
        "r2 threshold on test": test_accuracy_threshold,
        "accuracy": accuracy_score(y_test, test_predictions),
        "model saved" : model_saved,
        "Total Features" : len(X.columns),
        "Input Feartures Required" : list(X.columns),
        "session_put" : session_put
    }

The version of package 'scikit-learn' in the local environment is 1.3.1, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'joblib' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'joblib'. Your UDF might not work when the package version is different between the server and your local environment.


In [6]:
train_accuracy_threshold = 0.2
test_accuracy_threshold = 0.2
save = True
## Locally Trains the model
# local_train = dev_train_buf_model(session, "BUF_CUSTOMER_TABLE", train_accuracy_threshold, test_accuracy_threshold, save_model)
# local_train
## Calls the Stored Procedure from Snowflake to train the model
print(session.call('TRAIN_BUF_MODEL', 'BUF_CUSTOMER_TABLE', train_accuracy_threshold, test_accuracy_threshold, save))

{
  "Input Feartures Required": [
    "DUE_DATE_PAYMENT",
    "CREDIT_LIMIT_DATE",
    "STARTING_DATE_OF_RELATIONSHIP",
    "BILLING_CITY",
    "BILLING_STATE",
    "BILLING_COUNTRY",
    "BILLING_ZIP",
    "EMAIL_ADDRESS",
    "PHONE",
    "AVERAGE_DAILY_BALANCE",
    "CURRENT_BALANCE_OF_CUSTOMER",
    "TOTAL_PURCHASE_AMOUNT",
    "CREDIT_LIMIT",
    "INCOME_OF_THE_CUSTOMER",
    "CURRENTBANKBRANCHID",
    "CUSTOMERMONTHSONBOOKS",
    "AGE",
    "MONTHOFBIRTH_OF_CUSTOMER",
    "YEAROFBIRTH_OF_CUSTOMER"
  ],
  "Total Features": 19,
  "accuracy": 1,
  "model saved": true,
  "r2 score on teet": 1,
  "r2 score on train": 1,
  "r2 threshold on test": 0.2,
  "r2 threshold on train": 0.2,
  "session_put": [
    [
      "buf_model.joblib.gz",
      "buf_model.joblib.gz",
      105431,
      105440,
      "GZIP",
      "GZIP",
      "UPLOADED",
      ""
    ]
  ]
}


In [7]:
## Registers Stored Procedure for Training the model on Snowflake

# session.clear_imports()
# session.clear_packages()
# session.use_schema("USECASES")

# session.sproc.register(
#       func = dev_train_buf_model
#     , name = "TRAIN_BUF_MODEL"
#     , packages=["pandas", "scikit-learn", "snowflake-snowpark-python", "dill", "joblib"]
#     , is_permanent = True
#     , replace = True
#     , stage_location = '@ML_MODELS.USECASES.BUF_STAGE'
# )

## Create Vectorized User-Defined Function (UDF) using Batch API for inference
Batch UDF

In [8]:
from snowflake.snowpark.functions import col, call_udf, array_construct, udf

In [9]:
session.clear_imports()
session.clear_packages()

import cachetools
# Add trained model and Python packages from Snowflake Anaconda channel available on the server-side as UDF dependencies
session.add_import('@ML_MODELS.USECASES.BUF_STAGE/buf_model.joblib.gz')
session.add_packages('pandas', 'dill', 'joblib','cachetools','scikit-learn')

@cachetools.cached(cache={})
def load_model(filename):
    import joblib
    import dill
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = dill.load(file)
            return m

@udf(name='BATCH_PREDICT_BUF_SCORE',session=session,replace=True,is_permanent=True,stage_location='@ML_MODELS.USECASES.BUF_STAGE')
def batch_predict_roi(buf: list) -> float:
    columns = ['AGE', 'AVERAGE_DAILY_BALANCE', 'BILLING_CITY', 'BILLING_COUNTRY', 'BILLING_STATE', 'BILLING_ZIP', 'CREDIT_LIMIT', 'CREDIT_LIMIT_DATE', 'CURRENTBANKBRANCHID', 'CURRENT_BALANCE_OF_CUSTOMER', 'CUSTOMERMONTHSONBOOKS', 'DUE_DATE_PAYMENT', 'EMAIL_ADDRESS', 'INCOME_OF_THE_CUSTOMER', 'MONTHOFBIRTH_OF_CUSTOMER', 'PHONE', 'STARTING_DATE_OF_RELATIONSHIP', 'TOTAL_PURCHASE_AMOUNT', 'YEAROFBIRTH_OF_CUSTOMER']
    df = pd.DataFrame([buf], columns=columns)
    # inferencing
    model = load_model('buf_model.joblib.gz')
    return model.predict(df)

The version of package 'joblib' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'joblib'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'cachetools' in the local environment is 5.3.1, which does not fit the criteria for the requirement 'cachetools'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.1, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


In [3]:
from snowflake.snowpark.functions import col, call_udf, array_construct
table = session.table("BUF_CUSTOMER_TABLE")
result = table.select(*list(table.columns), call_udf("BATCH_PREDICT_BUF_SCORE", array_construct(
    col('AGE'), col('AVERAGE_DAILY_BALANCE'), col('BILLING_CITY'), col('BILLING_COUNTRY'), col('BILLING_STATE'), col('BILLING_ZIP'), col('CREDIT_LIMIT'), col('CREDIT_LIMIT_DATE'), col('CURRENTBANKBRANCHID'), col('CURRENT_BALANCE_OF_CUSTOMER'), col('CUSTOMERMONTHSONBOOKS'), col('DUE_DATE_PAYMENT'), col('EMAIL_ADDRESS'), col('INCOME_OF_THE_CUSTOMER'), col('MONTHOFBIRTH_OF_CUSTOMER'), col('PHONE'), col('STARTING_DATE_OF_RELATIONSHIP'), col('TOTAL_PURCHASE_AMOUNT'), col('YEAROFBIRTH_OF_CUSTOMER')
    )).as_("CUSTOMERSCORE"))

In [ ]:
df = result.toPandas()
df

### ############################################## DONE ########################################### #

# Custome Churn